# Exhibit 21 extraction

This notebook implements a model built on top of [layoutlmv3](https://huggingface.co/microsoft/layoutlmv3-base/tree/main)
from Exhibit 21 attachments to SEC-10k filings. These documents contain a list of all subsidiary companies owned by a filing
company.

## Load upstream assets and configuration
The following cell can be run interactively to set configuration and load upstream assets. When running the notebook in dagster, this cell will be replaced with assets from the dagster run and dagster run configuration.

### Config
- `layoutlm_uri`: If `None` the notebook will finetune layoutlm using `ex21_training_data`. If `layoutlm_uri` points to a valid model on the mlflow tracking server, the notebook will use the pre-trained model and perform inference on the validation set, logging validation metrics to a child run nested under the mlflow run associated with the pretrained model.

### Upstream assets
We are using dagster assets to construct training/validation data outside the notebook to allow for easy caching. These datasets are fairly compute intensive to create, so this is useful when iterating on the model using the same data.

NOTE: The notebook will load the most recent version of these assets, so to update the training/validation data you must rerun the dagster assets with desired configuration.

- `ex21_training_data`: Dataset containing labeled data produced in label-studio to train `layoutlm`
- `ex21_validation_set`: Labeled validation data describing expected inference output on validation filings
- `ex21_failed_parsing_metadata`: Metadata for any validation filings that couldn't be parsed (usually empty)
- `ex21_inference_dataset`: Parsed validation filings prepped for inference model

In [ ]:
import dagstermill

from mozilla_sec_eia.models.sec10k import defs

context = dagstermill.get_context(op_config={
    "layoutlm_training_run": "layoutlm-labeledv0.2",
})

ex21_training_data = defs.load_asset_value("ex21_training_data")

ex21_failed_parsing_metadata = defs.load_asset_value("ex21_failed_parsing_metadata")
ex21_inference_dataset = defs.load_asset_value("ex21_inference_dataset")
ex21_validation_set = defs.load_asset_value("ex21_validation_set")

## Train Layoutlmv3

### Define training metrics
The method `compute_metrics` will be used to score the model. It computes precision, recall, f1 score, and accuracy on bounding box labels output by `layoutlm`.

In [2]:
import os
from tempfile import TemporaryDirectory

import numpy as np
import pandas as pd

from mozilla_sec_eia.models.sec10k.utils.cloud import get_metadata_filename


def compute_metrics(p, metric, label_list, return_entity_level_metrics=False):
    """Compute metrics to train and evaluate the model on."""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[lab] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

#### Finetune Model
The next cell will use the functions defined in the previous section to actually construct a huggingface dataset from labeled data and finetune the `layoutlm` model. Model finetuning will only be run if configured to do so, otherwise a pretrained version will be used from the `mlflow` tracking server.

Model training contains several steps implemented below:
1. Use temporary path to convert filings to PDF's and stash labels
2. Use PDF's and labels to convert PDF's and labels to NER annotations
3. Construct huggingface dataset from NER annotations and split into train and test sets
4. Load pretrained model from huggingface
5. Finetune model on training data and evaluate on test data

In [3]:
import mlflow
from datasets import (
    Array2D,
    Array3D,
    Dataset,
    Features,
    Sequence,
    Value,
    load_metric,
)
from dotenv import load_dotenv
from transformers import (
    AutoProcessor,
    LayoutLMv3ForTokenClassification,
    Trainer,
    TrainingArguments,
)
from transformers.data.data_collator import default_data_collator

from mozilla_sec_eia.library.mlflow import configure_mlflow
from mozilla_sec_eia.models.sec10k.ex_21.data.common import (
    BBOX_COLS,
    LABELS,
    get_id_label_conversions,
)

load_dotenv()


configure_mlflow()
mlflow.set_experiment("exhibit21_extraction_test")


def _prepare_dataset(annotations, processor, label2id):
    """Put the dataset in its final format for training LayoutLM."""

    def _convert_ner_tags_to_id(ner_tags, label2id):
        return [int(label2id[ner_tag]) for ner_tag in ner_tags]

    images = annotations["image"]
    words = annotations["tokens"]
    boxes = annotations["bboxes"]
    # Map over labels and convert to numeric id for each ner_tag
    ner_tags = [
        _convert_ner_tags_to_id(ner_tags, label2id)
        for ner_tags in annotations["ner_tags"]
    ]

    encoding = processor(
        images,
        words,
        boxes=boxes,
        word_labels=ner_tags,
        truncation=True,
        padding="max_length",
    )

    return encoding

if (run_name := context.op_config["layoutlm_training_run"]) is not None:
    filter_string = f"attributes.run_name = '{run_name}'"
    run = mlflow.search_runs(filter_string=filter_string, output_format="list")[0]
    training_run_id = run.info.run_id
else:
    training_run_id = None

# Only finetune if configured to do so
if training_run_id is None:
    id2label, label2id = get_id_label_conversions(LABELS)
    # Change temp_dir to save training data locally for inspection
    # Cache/prepare training data
    dataset = Dataset.from_list(ex21_training_data)

    # Load pretrained model
    model = LayoutLMv3ForTokenClassification.from_pretrained(
        "microsoft/layoutlmv3-base", id2label=id2label, label2id=label2id
    )
    processor = AutoProcessor.from_pretrained(
        "microsoft/layoutlmv3-base", apply_ocr=False
    )

    # Prepare our train & eval dataset
    column_names = dataset.column_names
    features = Features(
        {
            "pixel_values": Array3D(dtype="float32", shape=(3, 224, 224)),
            "input_ids": Sequence(feature=Value(dtype="int64")),
            "attention_mask": Sequence(Value(dtype="int64")),
            "bbox": Array2D(dtype="int64", shape=(512, 4)),
            "labels": Sequence(feature=Value(dtype="int64")),
        }
    )
    dataset = dataset.map(
        lambda annotations: _prepare_dataset(annotations, processor, label2id),
        batched=True,
        remove_columns=column_names,
        features=features,
    )
    dataset.set_format("torch")
    split_dataset = dataset.train_test_split(test_size=0.2)
    train_dataset, eval_dataset = split_dataset["train"], split_dataset["test"]

    # Initialize our Trainer
    metric = load_metric("seqeval")
    training_args = TrainingArguments(
        max_steps=1000,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        learning_rate=1e-5,
        evaluation_strategy="steps",
        eval_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        output_dir="./layoutlm",
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=processor,
        data_collator=default_data_collator,
        compute_metrics=lambda p: compute_metrics(p, metric=metric, label_list=LABELS),
    )

    with mlflow.start_run() as training_run:
        # Train inside mlflow run. Mlflow will automatically handle logging training metrcis
        trainer.train()

        # Log finetuend model with mlflow
        model = {"model": trainer.model, "tokenizer": trainer.tokenizer}
        mlflow.transformers.log_model(
            model, artifact_path="layoutlm_extractor", task="token-classification"
        )
        training_run_id = training_run.info. run_id

## Model inference
Use the finetuned model to perform inference and evaluate on labeled validation data. First create a Huggingface `Pipeline` which wraps layoutlm with some custom pre/post processing steps.

In [4]:
import torch
from transformers import Pipeline, pipeline
from transformers.tokenization_utils_base import BatchEncoding

from mozilla_sec_eia.models.sec10k.ex_21.data.common import (
    get_flattened_mode_predictions,
)
from mozilla_sec_eia.models.sec10k.utils.layoutlm import (
    iob_to_label,
)


def separate_entities_by_row(entity_df):
    """Separate entities that span multiple rows and should be distinct.

    Sometimes LayoutLM groups multiple entities that span multiple rows
    into one entity. This function makes an attempt to break these out
    into multiple entities, by taking the average distance between rows
    and separating a grouped entity if the distance between y values
    is greater than the third quantile of y value spacing.
    """
    threshold = 1.0
    entity_df.loc[:, "line_group"] = entity_df.loc[:, "top_left_y"].transform(
        lambda y: (y // threshold).astype(int)
    )
    # Get the unique y-values for each line (group) per file
    line_positions = (
        entity_df.groupby(["line_group"])["top_left_y"].mean().reset_index()
    )
    # Calculate the difference between adjacent y-values (i.e., distance between lines)
    line_positions.loc[:, "y_diff"] = line_positions.loc[:, "top_left_y"].diff()
    # Filter out NaN values and take the mean of the valid distances
    y_diffs = line_positions["y_diff"].dropna()
    avg_y_diff = y_diffs.apply(np.floor).mean()
    # if an I labeled entity is more than avg_y_diff from it's previoius box then make it a B entity
    entity_df.loc[:, "prev_y"] = entity_df.loc[:, "top_left_y"].shift(1)
    entity_df.loc[:, "prev_iob"] = entity_df.loc[:, "iob_pred"].shift(1)

    # If the current prediction is an I label
    # and y distance exceeds the average y difference
    # update to a B label and make it the start of a new entity
    entity_df.loc[:, "iob_pred"] = np.where(
        (entity_df["iob_pred"].str[0] == "I")
        & ((entity_df["top_left_y"] - entity_df["prev_y"]) >= avg_y_diff),
        "B" + entity_df["iob_pred"].str[1:],  # Update to 'B'
        entity_df["iob_pred"],  # Keep as is
    )

    # Drop temporary columns
    return entity_df.drop(columns=["prev_y", "prev_iob"])

class LayoutLMInferencePipeline(Pipeline):
    """Pipeline for performing inference with fine-tuned LayoutLM."""

    def __init__(self, *args, **kwargs):
        """Initialize LayoutLMInferencePipeline."""
        super().__init__(*args, **kwargs)

    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, doc_dict):
        """Encode and tokenize model inputs."""
        image = doc_dict["image"]
        words = doc_dict["tokens"]
        boxes = doc_dict["bboxes"]
        encoding = self.tokenizer(
            image,
            words,
            boxes=boxes,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512,  # this is the maximum max_length
            stride=128,
            return_offsets_mapping=True,
            return_overflowing_tokens=True,
        )
        model_inputs = {}
        model_inputs["raw_encoding"] = encoding.copy()
        model_inputs["doc_dict"] = doc_dict
        model_inputs["offset_mapping"] = encoding.pop("offset_mapping")
        model_inputs["sample_mapping"] = encoding.pop("overflow_to_sample_mapping")
        # TODO: do we actually need to make these into ints?
        encoding["input_ids"] = encoding["input_ids"].to(torch.int64)
        encoding["attention_mask"] = encoding["attention_mask"].to(torch.int64)
        encoding["bbox"] = encoding["bbox"].to(torch.int64)
        encoding["pixel_values"] = torch.stack(encoding["pixel_values"])
        model_inputs["encoding"] = encoding
        return model_inputs

    def _forward(self, model_inputs):
        # encoding is passed as a UserDict in the model_inputs dictionary
        # turn it back into a BatchEncoding
        encoding = BatchEncoding(model_inputs["encoding"])
        if torch.cuda.is_available():
            encoding.to("cuda")
            self.model.to("cuda")
        # since we're doing inference, we don't need gradient computation
        with torch.no_grad():
            output = self.model(**encoding)
            return {
                "logits": output.logits,
                "predictions": output.logits.argmax(-1).squeeze().tolist(),
                "raw_encoding": model_inputs["raw_encoding"],
                "doc_dict": model_inputs["doc_dict"],
            }

    def postprocess(self, output_dict):
        """Return logits, model predictions, and the extracted dataframe."""
        output_df = self.extract_table(output_dict)
        output_dict["output_df"] = output_df
        return output_dict

    def extract_table(self, output_dict):
        """Extract a structured table from a set of inference predictions.

        This function essentially works by stacking bounding boxes and predictions
        into a dataframe and going from left to right and top to bottom. Then, every
        every time a new subsidiary entity is encountered, it assigns a new group or
        "row" to that subsidiary. Next, location and ownership percentage words/labeled
        entities in between these subsidiary groups are assigned to a subsidiary row/group.
        Finally, this is all formatted into a dataframe with an ID column from the original
        filename and a basic cleaning function normalizes strings.
        """
        # TODO: when model more mature, break this into sub functions to make it
        # clearer what's going on
        predictions = output_dict["predictions"]
        encoding = output_dict["raw_encoding"]
        doc_dict = output_dict["doc_dict"]

        token_boxes_tensor = encoding["bbox"].flatten(start_dim=0, end_dim=1)
        predictions_tensor = torch.tensor(predictions)
        mode_predictions = get_flattened_mode_predictions(
            token_boxes_tensor, predictions_tensor
        )
        token_boxes = encoding["bbox"].flatten(start_dim=0, end_dim=1).tolist()
        predicted_labels = [
            self.model.config.id2label[pred] for pred in mode_predictions
        ]
        simple_preds = [iob_to_label(pred).lower() for pred in predicted_labels]

        df = pd.DataFrame(data=token_boxes, columns=BBOX_COLS)
        df.loc[:, "iob_pred"] = predicted_labels
        df.loc[:, "pred"] = simple_preds
        invalid_mask = (
            (df["top_left_x"] == 0)
            & (df["top_left_y"] == 0)
            & (df["bottom_right_x"] == 0)
            & (df["bottom_right_y"] == 0)
        )
        df = df[~invalid_mask]
        # we want to get actual words on the dataframe, not just subwords that correspond to tokens
        # subwords from the same word share the same bounding box coordinates
        # so we merge the original words onto our dataframe on bbox coordinates
        words_df = pd.DataFrame(data=doc_dict["bboxes"], columns=BBOX_COLS)
        words_df.loc[:, "word"] = doc_dict["tokens"]
        df = df.merge(words_df, how="left", on=BBOX_COLS).drop_duplicates(
            subset=BBOX_COLS + ["pred", "word"]
        )
        df = df.sort_values(by=["top_left_y", "top_left_x"])
        # rows that are the first occurrence in a new group (subsidiary, loc, own_per)
        # should always have a B entity label. Manually override labels so this is true.
        first_in_group_df = df[
            (df["pred"].ne(df["pred"].shift())) & (df["pred"] != "other")
        ]
        first_in_group_df.loc[:, "iob_pred"] = (
            "B" + first_in_group_df["iob_pred"].str[1:]
        )
        df.update(first_in_group_df)
        # filter for just words that were labeled with non "other" entities
        entities_df = df[df["pred"] != "other"]
        # boxes that have the same group label but are on different rows
        # should be updated to have two different B labels

        entities_df = entities_df.groupby("pred").apply(separate_entities_by_row, include_groups=False)
        entities_df = entities_df.reset_index("pred").sort_index()
        # merge B and I entities to form one entity group
        # (i.e. "B-Subsidiary" and "I-Subsidiary" become just "subsidiary"), assign a group ID
        entities_df["group"] = (entities_df["iob_pred"].str.startswith("B-")).cumsum()
        grouped_df = (
            entities_df.groupby(["group", "pred"])["word"]
            .apply(" ".join)
            .reset_index()[["pred", "word"]]
        )
        # assign a new row every time there's a new subsidiary
        grouped_df["row"] = (grouped_df["pred"].str.startswith("subsidiary")).cumsum()
        output_df = grouped_df.pivot_table(
            index="row", columns="pred", values="word", aggfunc=lambda x: " ".join(x)
        ).reset_index()
        if output_df.empty:
            return output_df
        output_df.loc[:, "id"] = doc_dict["id"]
        return output_df

Next, wrap the `LayoutLMInferencePipeline` in an `mlflow` `pyfunc` model, which handles loading the pretrained model and managing inputs/outputs.

In [1]:
from PIL import Image

from mozilla_sec_eia.models.sec10k.entities import (
    Ex21CompanyOwnership,
    Sec10kExtractionMetadata,
)
from mozilla_sec_eia.models.sec10k.ex_21.ex21_validation_helpers import (
    clean_extracted_df,
)

# If a model was trained in this notebook, use it. Otherwise, use
model_uri = f"runs:/{training_run_id}/layoutlm_extractor"
model_info = mlflow.models.get_model_info(model_uri)

def _get_data(dataset):
    yield from dataset

def _fill_known_nulls(df):
    """Fill known nulls in location and own per column.

    Fill with known values from rows with same subsidiary.
    If an extracted Ex. 21 table looks like the following:

    subsidiary   loc       own_per
    Company A    NaN       NaN
    Company A    Delaware  50

    Then fill in the first row with location and ownership
    percentage from the second row.
    """
    if "own_per" in df:
        df["own_per"] = df.groupby(["id", "subsidiary"])["own_per"].transform(
            lambda group: group.ffill()
        )
    if "loc" in df:
        df["loc"] = df.groupby(["id", "subsidiary"])["loc"].transform(
            lambda group: group.ffill()
        )
    return df

class Ex21Extractor(mlflow.pyfunc.PythonModel):
    """Create an mlflow pyfunc model to perform full EX21 extraction."""
    def load_context(self, context):
        """Load pretrained model."""
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
        self.model_components = mlflow.transformers.load_model(
            context.artifacts["model_components"], return_type="components"
        )

    def predict(self, context, model_input: pd.DataFrame, params=None):
        """Use pretrained model and inference pipeline to perform inference."""
        # Convert dataframe to pyarrow Dataset
        model_input["image"] = model_input.apply(
            lambda row: Image.frombytes(
                row["mode"], (row["width"], row["height"]), row["image"]
            ),
            axis=1,
        )
        dataset = Dataset.from_list(model_input.drop(["mode", "width", "height"], axis=1).to_dict("records"))

        # TODO: figure out device argument
        pipe = pipeline(
            "token-classification",
            model=self.model_components["model"],
            tokenizer=self.model_components["tokenizer"],
            pipeline_class=LayoutLMInferencePipeline,
            device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        )

        logits = []
        predictions = []
        all_output_df = Ex21CompanyOwnership.example(size=0)
        extraction_metadata = Sec10kExtractionMetadata.example(size=0)
        for output_dict in pipe(_get_data(dataset)):
            logits.append(output_dict["logits"])
            predictions.append(output_dict["predictions"])
            output_df = output_dict["output_df"]
            if not output_df.empty:
                filename = get_metadata_filename(output_df["id"].iloc[0])
                extraction_metadata.loc[filename, ["success"]] = True
            all_output_df = pd.concat([all_output_df, output_df])
        all_output_df.columns.name = None
        all_output_df = clean_extracted_df(all_output_df)
        all_output_df = _fill_known_nulls(all_output_df)
        all_output_df = all_output_df[["id", "subsidiary", "loc", "own_per"]].drop_duplicates()
        all_output_df = all_output_df.reset_index(drop=True)
        outputs_dict = {
            "all_output_df": all_output_df,
            "logits": logits,
            "predictions": predictions,
        }
        return extraction_metadata, outputs_dict

# Save model to local temp dir with artifacts, then reload for evaluation
with TemporaryDirectory() as tmp_dir:
    mlflow.pyfunc.save_model(
        path=tmp_dir,
        python_model=Ex21Extractor(),
        artifacts={"model_components": model_uri},
    )
    ex21_extraction_model = mlflow.pyfunc.load_model(tmp_dir)

NameError: name 'training_run_id' is not defined

### Model Evaluation
Now the full extraction model can be evaluated using labeled validation data and logged to `mlflow`. The `mlflow` run used to evaluate and log the inference model will be created as a nested child run to the run used to train `layoutlm`. This setup allows multiple versions/configurations of inference to be associated with a single version of `layoutlm`, creating a clean organizational structure for testing the base model and inference logic separately.

#### Validate model
Finally, run the full model on the validation set and log metrics to mlflow. The logged metrics/model will appear in a nested run below the training run used for the current version of the model.

In [ ]:
from mlflow.models import infer_signature

from mozilla_sec_eia.models.sec10k.ex_21.ex21_validation_helpers import (
    ex21_validation_metrics,
)

with mlflow.start_run(parent_run_id=model_info.run_id, nested=True):
    metadata, outputs_dict = ex21_extraction_model.predict(ex21_inference_dataset.copy())
    extracted = outputs_dict["all_output_df"]
    metadata = pd.concat([ex21_failed_parsing_metadata, metadata])

    jaccard_df, prec_recall_df, incorrect_filenames, metrics = ex21_validation_metrics(extracted, ex21_validation_set)
    mlflow.log_metrics(metrics)
    mlflow.pyfunc.log_model(
        "exhibit21_extractor",
        python_model=Ex21Extractor(),
        artifacts={"model_components": model_uri},
        signature=infer_signature(ex21_inference_dataset, extracted), # NOTE: model returns a second dataframe with metadata, but mlflow only supports one in signature
    )
    mlflow.log_table(extracted, "extracted_data.json")
    mlflow.log_table(metadata, "extraction_metadata.json")

2024/10/16 17:11:53 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/home/zach/mambaforge/envs/mozilla-sec-eia/lib/python3.11/site-packages/transformers/modeling_utils.py:1101: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/zach/mambaforge/envs/mozilla-sec-eia/lib/python3.11/site-packages/transformers/modeling_utils.py:1101: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/zach/mambaforge/envs/mozilla-sec-eia/lib/python3.11/site-packages/transformers/modeling_utils.py:1101: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/zach/mambaforge/envs/mozilla-sec-eia/lib/python3.11/site-packages/transformers/modeling_utils.py:1101: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/zach/mam

2024/10/16 17:12:58 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-08-29; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'catalystcoop-mozilla-sec-eia'}
2024/10/16 17:12:58 WARNING mlflow.utils.requirements_utils: Found catalystcoop-mozilla-sec-eia version (0.1.dev353+gdf5fe0d.d20241011) contains a local version label (+gdf5fe0d.d20241011). MLflow logged a pip requirement for this package as 'catalystcoop-mozilla-sec-eia==0.1.dev353' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
